# 通用算法库

In [1]:
%matplotlib
import numpy as np
import matplotlib.pyplot as plt

Using matplotlib backend: Qt5Agg


## 1 连续值离散化
针对一个连续变量c：[DOWN,UP]，以b为间隔将其转换成离散化后的值d：[0,N+1]。  
若c取值在区间外，则d = N+1。  
该功能可用于强化学习中构建Q表时，将连续状态值映射为Q表的矩阵索引。  

In [2]:
DOWN, UP = -5, 5
b = 0.5
def map_continuous_discretization(c, down, up, b):
    if not (c > down and c < up):
        c = up
    return int((c - down) / b)
d = map_continuous_discretization(2.2, DOWN, UP, b)
print(d)

14


In [3]:
def map_continuous_discretization_np(c, down, up, b):
    '''numpy版本，自带digitize函数'''
    bins = np.arange(down, up, b)
    return np.digitize(c, bins)
d = map_continuous_discretization_np(2.2, -5, 5, 0.5)
print(d)

15


## 2 参数可视化——渐变色表示法
在可视化某些参数的时候希望通过色彩表现区分。但仅仅将RGB的某一个通道用该参数表示，辨识度并不高。所以需要建立一个渐变色范围表。

In [4]:
def gradient_color():
    """绘制三张渐变色图像"""
    img1 = np.ones([255,255,3], dtype=np.uint8)
    for x in range(img1.shape[0]):
        for y in range(img1.shape[1]):
            img1[x,:,0] = 255-x
            img1[x,y,1] = int(x/2 + y/2)
            img1[:,y,2] = 255-y
    img2 = np.ones([255,255,3], dtype=np.uint8)
    for x in range(img2.shape[0]):
        for y in range(img2.shape[1]):
            img2[x,:,0] = x
            img2[x,y,1] = y
            img2[x,:,2] = 255-x
    img3 = np.ones([255,255,3], dtype=np.uint8)
    for x in range(img3.shape[0]):
        img3[x,:,0] = x
        img3[x,:,1] = 0
        img3[x,:,2] = 255-x
    plt.subplot(131);    plt.imshow(img1)
    plt.subplot(132);    plt.imshow(img2)
    plt.subplot(133);    plt.imshow(img3)
    plt.show()
gradient_color()

## 3 one-hot分布
在对多个数据加权处理时，权重往往和分布有关。  
对一列数据a:[x1,x2,...xn]，转换成one-hot分布，其中hot值的位置是a中最大值的位置。  
P[np.argmax(a)] = 1  

In [5]:
def one_hot():
    a = np.random.randint(0, 100, 10)
    E = np.zeros(a.shape)
    E[np.argmax(a)] = 1
    print(a)
    print(E)
one_hot()

[48  7 16 10 50 45 41  9 78 72]
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]


## 4 $\epsilon-$greedy算法
强化学习中根据Q表选择行为a往往会用到$\epsilon-$greedy算法。  
以$\epsilon$的概率均匀地从行为空间中选择，以$(1-\epsilon)$的概率从行为空间中选择价值最大的a，这就会用到one-hot分布和均匀分布。

In [6]:
def epsilon_greedy(Qs,A,epsilon=0.3):
    """Qs：Q表在S=s的这一行  [M]
       A：行为空间   [M]
    """
    M = A.shape
    E = np.zeros(M)                   ## one-hot 分布
    E[np.argmax(Qs)] = 1
    P = np.ones(M)                    ## 均匀分布
    P = P / M
    Prop = epsilon * P + (1 - epsilon) * E
    a = np.random.choice(A, p=Prop)
    return a
Qs = 10 * np.random.rand(5)
A = np.arange(5)
a = epsilon_greedy(Qs, A, epsilon=0.3)
print(Qs); print(A); print(a)

[5.09424224 7.4826224  3.25789642 0.72548996 0.48557425]
[0 1 2 3 4]
1


## 5 栅格化处理图像  
对任意大小的图像，以kernel-size为边长的正方形块栅格化，图像多出的边界默认裁掉。

In [ ]:
def conv(img, kernel_size):
    H = img.shape[0];  nh = H // kernel_size
    W = img.shape[1];  nw = W // kernel_size
    Id = 0
    for ih in range(nh):
        for jw in range(nw):
            kernel = img[ih*kernel_size : (ih+1)*kernel_size, jw*kernel_size : (jw+1)*kernel_size]
            num += 1